# 

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 24 15:44:46 2025

@author: gruffalo
"""

# %% Set working directory
import os
os.chdir(r'/home/pinky/Documents/PrgGithub/Ella/Python/data_SophieBagur')

# import scipy.io
from load_save_results import ( 
    load_results
    )
from joblib import load
from preprocess_sort_data import (
    rebin_mice_data,
    normalize_data
    )
from analyse_data import (
    spike_count_all_mice,
    )
from preprocess_linear_model import (
    combine_mouse_data
    )
from analyse_ln_corrected_TC import (
    predict_all_neurons,
    correct_predictions,
    correct_combined_data
    )
from plot_ln_corrected_TC import (
    # plot_predictions_trace,
    plot_tuning_curve,
    plot_tuning_curve_heatmap,
    plot_mean_correction
    )
from fit_linear_nonlinear_model import (
    process_neuron
    )
from save_plots import (
    save_plot_as_svg
    )

In [3]:
# %% Load data and results

load_path = r'/media/DataMOBsRAIDN/ProjectEmbReact/Data_ella/'
figures_directory = load_path + 'figures/corrected_TC'

maze_denoised_data = load_results(load_path + 'maze_denoised_data.pkl')
maze_spike_times_data = load_results(load_path + 'maze_spike_times_data.pkl')

# Rebin data
new_bin_size = 0.6
maze_rebinned_data = rebin_mice_data(maze_denoised_data, ['BreathFreq', 'Heartrate', 'Accelero', 'Speed', 'LinPos'], new_bin_size)
maze_rebinned_normalized_data = normalize_data(maze_rebinned_data, ['BreathFreq', 'Heartrate', 'Accelero', 'Speed', 'LinPos'])

# Compute spike counts
spike_counts = spike_count_all_mice(maze_rebinned_data, maze_spike_times_data)

# results_ln_loaded = load_results(load_path + 'results_ln_model_rs30.pkl')
results_ln_loaded = load(load_path + 'all_results_ln_model_rs30.joblib')

combined_data = combine_mouse_data(spike_counts, 
                                   maze_rebinned_normalized_data, drop_na=True)

combined_data_raw = combine_mouse_data(spike_counts, maze_rebinned_data, drop_na=True)

# %% Compute the predictions on all the dataset

predicted = predict_all_neurons(results_ln_loaded, combined_data, ['motion'])

# plot_predictions_trace(predicted, 'Mouse508', 'Neuron_16', 'motion')

# %% Substract the predictions to correct the data

corrected_motion = correct_predictions(predicted, 'motion')

Data loaded successfully from /media/DataMOBsRAIDN/ProjectEmbReact/Data_ella/maze_denoised_data.pkl.
Data loaded successfully from /media/DataMOBsRAIDN/ProjectEmbReact/Data_ella/maze_spike_times_data.pkl.
Column 'Heartrate' contains only NaN values. Skipping...


/home/pinky/anaconda3/envs/sophie_basic/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.6.0 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/pinky/anaconda3/envs/sophie_basic/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.0 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Combining data for mouse Mouse509...
Excluded columns: Mouse509, []
Combining data for mouse Mouse490...
Excluded columns: Mouse490, ['Heartrate']
Combining data for mouse Mouse508...
Excluded columns: Mouse508, []
Combining data for mouse Mouse507...
Excluded columns: Mouse507, []
Combining data for mouse Mouse512...
Excluded columns: Mouse512, ['Heartrate']
Combining data for mouse Mouse510...
Excluded columns: Mouse510, []
Combining data for mouse Mouse514...
Excluded columns: Mouse514, ['Heartrate']
Combining data for mouse Mouse509...
Excluded columns: Mouse509, []
Combining data for mouse Mouse490...
Excluded columns: Mouse490, ['Heartrate']
Combining data for mouse Mouse508...
Excluded columns: Mouse508, []
Combining data for mouse Mouse507...
Excluded columns: Mouse507, []
Combining data for mouse Mouse512...
Excluded columns: Mouse512, ['Heartrate']
Combining data for mouse Mouse510...
Excluded columns: Mouse510, []
Combining data for mouse Mouse514...
Excluded columns: Mouse5

In [73]:
import numpy as np
#corrected_motion[0]['Correction']
#np.save("combined_data",combined_data)
#np.save("corrected_motion",corrected_motion)
data = corrected_motion
# Extract Mouse IDs as integers
MouseID = np.array([int(d['Mouse_ID'].replace('Mouse', '')) for d in data])

# Extract Neuron numbers as integers
NeuronNum = np.array([int(d['Neuron_ID'].replace('Neuron_', '')) for d in data])

# Stack Corrections into a single 2D NumPy array
# Find the max length
max_len = max(len(d['Correction']) for d in data)

# Pad with NaNs
CorrectedData = np.full((len(data), max_len), np.nan)
for i, d in enumerate(data):
    length = len(d['Correction'])
    CorrectedData[i, :length] = d['Correction']

dfs = list(combined_data.values())   # list of DataFrames

# Get the maximum number of timebins
max_len = max(len(df) for df in dfs)

# Allocate padded array
TimeBinsArray = np.full((len(dfs), max_len), np.nan)

for i, df in enumerate(dfs):
    timebins = df.index.values         # index contains the timebins
    TimeBinsArray[i, :len(timebins)] = timebins


  
np.save("corrected_motion_timebins",TimeBinsArray)
np.save("corrected_motion_data",CorrectedData)
np.save("corrected_motion_mouseID",MouseID)
np.save("corrected_motion_neuronNum",NeuronNum)

In [72]:
dfs = list(combined_data.values())   # list of DataFrames

# Get the maximum number of timebins
max_len = max(len(df) for df in dfs)

# Allocate padded array
TimeBinsArray = np.full((len(dfs), max_len), np.nan)

for i, df in enumerate(dfs):
    timebins = df.index.values         # index contains the timebins
    TimeBinsArray[i, :len(timebins)] = timebins




(7, 4746)


# 

In [47]:



# Allocate a padded NumPy array
TimeBinsArray = np.full((len(dfs), max_len), np.nan)

# Fill each row with that mouse's timebins
for i, df in enumerate(dfs):
    timebins = df['timebins'].values  # NumPy array
    TimeBinsArray[i, :len(timebins)] = timebins

# Also extract the mouse IDs to keep track
mouse_ids = list(combined_data.keys())

print(TimeBinsArray.shape)
print(mouse_ids)

pandas.core.frame.DataFrame